In [0]:
import os

storage_account_name = "dota2lakehousenew"
storage_account_key = os.environ.get("AZURE_STORAGE_KEY")
container_name = "data"

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

print("Storage account has been configured!")

In [0]:
gold_path = "abfss://data@dota2lakehousenew.dfs.core.windows.net/gold"

player_stats = spark.read.format("delta").load(f"{gold_path}/player_stats")
player_stats.createOrReplaceTempView("gold_player_stats")
print("gold_player_stats table has been created")

hero_stats = spark.read.format("delta").load(f"{gold_path}/hero_stats")
hero_stats.createOrReplaceTempView("gold_hero_stats")
print("gold_hero_stats table has been created")

daily_stats = spark.read.format("delta").load(f"{gold_path}/daily_stats")
daily_stats.createOrReplaceTempView("gold_daily_stats")
print("gold_daily_stats table has been created")

ml_features = spark.read.format("delta").load(f"{gold_path}/ml_features")
ml_features.createOrReplaceTempView("gold_ml_features")
print("gold_ml_features table has been created")

In [0]:
%sql
SELECT 
    account_id,
    total_kills, 
    total_deaths,
    total_assists,
    ROUND(avg_kda, 2) as kda_ratio
FROM 
    gold_player_stats
WHERE 
    total_matches > 5
ORDER BY 
    avg_kda DESC
LIMIT 10;

In [0]:
%sql
SELECT 
    hero_id,
    times_played as total_picks,
    wins as total_wins,
    ROUND(win_rate * 100, 2) as win_rate_percentage
FROM 
    gold_hero_stats
ORDER BY 
    times_played DESC
LIMIT 10;

In [0]:
%sql
SELECT 
    match_date,
    match_count,
    ROUND(avg_duration / 60, 2) as avg_duration_minutes
FROM 
    gold_daily_stats
ORDER BY 
    match_date DESC
LIMIT 10;

In [0]:
%sql
SELECT 
    hero_id,
    ROUND(avg_gpm, 0) as avg_gpm,
    ROUND(avg_xpm, 0) as avg_xpm
FROM 
    gold_hero_stats
ORDER BY 
    avg_gpm DESC
LIMIT 10;

In [0]:
%sql
SELECT 
    hero_id,
    ROUND(avg_kills, 2) as avg_kills,
    ROUND(avg_assists, 2) as avg_assists,
    times_played
FROM 
    gold_hero_stats
ORDER BY 
    avg_kills DESC
LIMIT 10;